### Preprocessing Eclipse Bug reports

#### Description of attributes
    Size of data: (103805, 13)
    Bug ID: Unique Identifier
    Important attributes: Product, Component, Assignee, Summary, Changed (possibly the frequency it has been changed with difference between opened and changed), Summary, Severity, Priority:
    Unimportant attributes: Status and Resolution. [We will not know them for the test samples]
    
    Summary has 103802 values i.e. 3 missing values will have to investigate them.


### Fixing of data formatting in the csv
    Some rows have shifted right by a few columns due the the data delimiting in the csv like rows 3613, 258
    That has been fixed and a new file total_fixed.csv has been saved

In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('../data/processed/train_processed.csv')

In [16]:
df.shape

(83044, 41)

In [17]:
import numpy as np

In [18]:
len(np.unique(df['Bug ID']))

83044

In [19]:
df.drop('Bug ID',axis=1,inplace=False).describe()

,TMP1,TMP2,TMP3,TMP4,TMP5,TMP6,TMP7,TMP8,TMP9,TMP10,...,PRO12.1,PRO14,PRO15,PRO16,PRO17,PRO18,PRO19,PRO20,PRO21,PRO22
count,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,...,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000,83044.000000
mean,445.465524,229.932289,337.405556,1384.912492,706.768520,1044.634603,193.802009,101.649066,147.428496,270.287306,...,7150.058595,3511.744340,5340.331704,0.057358,0.106323,0.767588,0.042338,0.026392,3.140094,3.005539
std,422.592240,281.797592,356.454094,641.655154,556.997943,643.107237,421.593892,254.344225,337.157303,421.749476,...,8124.428881,4928.031965,6637.592758,0.047891,0.053526,0.112600,0.041118,0.061300,0.358185,0.167629
min,21.000000,1.000000,1.000000,393.000000,1.000000,1.000000,2.000000,1.000000,1.000000,7.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.267045,0.000000,0.000000,2.000000,2.000000
25%,233.000000,50.000000,159.000000,862.000000,173.000000,618.000000,63.000000,15.000000,40.000000,110.000000,...,1326.000000,342.000000,713.000000,0.030389,0.056599,0.696059,0.013362,0.003805,3.000000,3.000000
50%,331.000000,170.000000,255.000000,1216.000000,611.000000,929.000000,96.000000,47.000000,73.000000,164.000000,...,3832.000000,1324.000000,2507.500000,0.047474,0.111552,0.775334,0.032736,0.009953,3.000000,3.000000
75%,491.000000,300.000000,404.000000,1805.000000,1052.000000,1446.000000,155.000000,92.000000,124.000000,261.000000,...,9377.000000,4129.000000,6635.250000,0.071726,0.139819,0.843322,0.058060,0.014553,3.000000,3.000000
max,2865.000000,2057.000000,2865.000000,3401.000000,2458.000000,3401.000000,2761.000000,1970.000000,2761.000000,2796.000000,...,30084.000000,20297.000000,30070.000000,0.345515,0.666667,1.000000,0.333333,0.540284,5.000000,5.000000


In [20]:
df.loc[df.Summary.isnull(),:]

,Bug ID,Severity,Priority,Summary,TMP1,TMP2,TMP3,TMP4,TMP5,TMP6,...,PRO12.1,PRO14,PRO15,PRO16,PRO17,PRO18,PRO19,PRO20,PRO21,PRO22
3819,1719,normal,P3,NaN,2356,1599,1853,2356,1599,1853,...,212,113,143,0.202830,0.240566,0.500000,0.056604,0.000000,3,3
9459,65246,normal,P3,NaN,857,610,731,2784,2015,2402,...,16183,10522,12895,0.061237,0.134833,0.735463,0.058889,0.009578,3,3
25374,38524,normal,P3,NaN,225,141,177,836,463,576,...,2756,1979,2492,0.054064,0.106676,0.720972,0.050435,0.067852,3,3


In [21]:
df.iloc[[257,258],:]

,Bug ID,Severity,Priority,Summary,TMP1,TMP2,TMP3,TMP4,TMP5,TMP6,...,PRO12.1,PRO14,PRO15,PRO16,PRO17,PRO18,PRO19,PRO20,PRO21,PRO22
257,209173,normal,P3,Configuration Problem Win2k at using a Eclipse...,155,102,123,572,393,469,...,668,498,589,0.000000,0.000000,0.995509,0.004491,0.000000,4,3
258,36684,normal,P3,BIDI: LTR-oriented MessageBox can't be created...,161,100,129,694,419,535,...,2597,1882,2355,0.055834,0.111282,0.710435,0.050828,0.071621,3,3


### Text preprocessing according to the Paper

1. Tokenization - word level
2. Stop-Word removal - 30 stop words: TIFIDF has that feature
3. Stemming - Porter Stemming.

This will be done only on the Summary attribute as others are categorical attributes.


In [22]:
df.columns

Index([u'Bug ID', u'Severity', u'Priority', u'Summary', u'TMP1', u'TMP2',
       u'TMP3', u'TMP4', u'TMP5', u'TMP6', u'TMP7', u'TMP8', u'TMP9', u'TMP10',
       u'TMP11', u'TMP12', u'AUT1', u'AUT2', u'AUT3', u'PRO1', u'PRO2',
       u'PRO3', u'PRO4', u'PRO5', u'PRO6', u'PRO7', u'PRO8', u'PRO9', u'PRO10',
       u'PRO11', u'PRO12', u'PRO12.1', u'PRO14', u'PRO15', u'PRO16', u'PRO17',
       u'PRO18', u'PRO19', u'PRO20', u'PRO21', u'PRO22'],
      dtype='object')

In [23]:
df.dropna(inplace=True)

In [24]:
df.isnull().sum()

Bug ID      0
Severity    0
Priority    0
Summary     0
TMP1        0
TMP2        0
TMP3        0
TMP4        0
TMP5        0
TMP6        0
TMP7        0
TMP8        0
TMP9        0
TMP10       0
TMP11       0
TMP12       0
AUT1        0
AUT2        0
AUT3        0
PRO1        0
PRO2        0
PRO3        0
PRO4        0
PRO5        0
PRO6        0
PRO7        0
PRO8        0
PRO9        0
PRO10       0
PRO11       0
PRO12       0
PRO12.1     0
PRO14       0
PRO15       0
PRO16       0
PRO17       0
PRO18       0
PRO19       0
PRO20       0
PRO21       0
PRO22       0
dtype: int64

In [25]:
df.drop(['Bug ID'], axis=1, inplace=True)

In [26]:
df.reset_index(inplace=True, drop=True)

In [27]:
categorical_cols = ['Severity', 'PRO1', 'PRO12']
other_cols = list(df.columns)
for c in categorical_cols:
    other_cols.remove(c)
other_cols.remove('Priority')
other_cols.remove('Summary')

In [28]:
print categorical_cols
print (other_cols)

['Severity', 'PRO1', 'PRO12']
['TMP1', 'TMP2', 'TMP3', 'TMP4', 'TMP5', 'TMP6', 'TMP7', 'TMP8', 'TMP9', 'TMP10', 'TMP11', 'TMP12', 'AUT1', 'AUT2', 'AUT3', 'PRO2', 'PRO3', 'PRO4', 'PRO5', 'PRO6', 'PRO7', 'PRO8', 'PRO9', 'PRO10', 'PRO11', 'PRO12.1', 'PRO14', 'PRO15', 'PRO16', 'PRO17', 'PRO18', 'PRO19', 'PRO20', 'PRO21', 'PRO22']


### Null values removed and only important features are kept

In [29]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import re

tokenizer = RegexpTokenizer(r'\w+')
stopwords = set(nltk.corpus.stopwords.words('english'))
stemmer = PorterStemmer()

In [30]:
# tokenizer.tokenize('Eighty-seven miles to go, yet.  Onward!')
def tokenize_stop_stem(text):
    try:
        tokens = tokenizer.tokenize(text)
        # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation) and stem
        filtered_tokens = []
        for token in tokens:
            token = token.lower()
            if token not in stopwords:
                if not re.search('[0-9]', token):
                    try:
                        token = stemmer.stem(token)
                        filtered_tokens.append(token)
                    except UnicodeDecodeError:
                        print 'illeagal token ignored:',token
                        pass
    except UnicodeDecodeError:
        print 'illeagal token ignored:',token
        pass
    return filtered_tokens

### Run sklearn countvectorizer

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
countVect = CountVectorizer(input='content',lowercase=False, max_features=395996, tokenizer=tokenize_stop_stem, decode_error='ignore')

In [33]:
countVector = countVect.fit_transform(df.Summary)

In [34]:
countVector

<83041x15905 sparse matrix of type '<type 'numpy.int64'>'
	with 467339 stored elements in Compressed Sparse Row format>

In [35]:
countVector.shape

(83041, 15905)

### Summary attribute has been preprocessed. Get dummy variables and join them as CSR and join with countvectors

In [36]:
dummy_df = pd.get_dummies( df[categorical_cols] )

In [37]:
dummy_df.head()

,Severity_blocker,Severity_critical,Severity_enhancement,Severity_major,Severity_minor,Severity_normal,Severity_trivial,PRO1_Equinox,PRO1_JDT,PRO1_PDE,...,PRO12_SWT,PRO12_Scripting,PRO12_Search,PRO12_Team,PRO12_Text,PRO12_UI,PRO12_Update (deprecated - use RT>Equinox>p2),PRO12_User Assistance,PRO12_WebDAV,PRO12_Website
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [38]:
from scipy import sparse

In [39]:
dummy_sparse = sparse.csr_matrix(dummy_df.values)

In [40]:
print df.columns

Index([u'Severity', u'Priority', u'Summary', u'TMP1', u'TMP2', u'TMP3',
       u'TMP4', u'TMP5', u'TMP6', u'TMP7', u'TMP8', u'TMP9', u'TMP10',
       u'TMP11', u'TMP12', u'AUT1', u'AUT2', u'AUT3', u'PRO1', u'PRO2',
       u'PRO3', u'PRO4', u'PRO5', u'PRO6', u'PRO7', u'PRO8', u'PRO9', u'PRO10',
       u'PRO11', u'PRO12', u'PRO12.1', u'PRO14', u'PRO15', u'PRO16', u'PRO17',
       u'PRO18', u'PRO19', u'PRO20', u'PRO21', u'PRO22'],
      dtype='object')


In [41]:
other_features = df[other_cols]

In [42]:
for i in range(len(other_cols)):
    try:
        test = other_features.iloc[:,i].astype(float)
    except:
        print "problem is in column - ", i, other_cols[i]

In [43]:
other_sparse = sparse.csr_matrix(other_features.values)

In [44]:
inputDF = sparse.hstack((dummy_sparse,countVector),format="csr")

In [45]:
inputDF = sparse.hstack((other_features,countVector),format="csr")

In [55]:
print inputDF.shape

(83041, 15940)


In [47]:
print countVector[0]

  (0, 1780)	1
  (0, 6944)	1
  (0, 3526)	1
  (0, 7250)	1


In [48]:
print other_sparse[0]

  (0, 0)	823.0
  (0, 1)	614.0
  (0, 2)	712.0
  (0, 3)	2245.0
  (0, 4)	1642.0
  (0, 5)	1955.0
  (0, 6)	121.0
  (0, 7)	78.0
  (0, 8)	100.0
  (0, 9)	242.0
  (0, 10)	172.0
  (0, 11)	203.0
  (0, 12)	3.0
  (0, 13)	3.0
  (0, 14)	715.0
  (0, 15)	24034.0
  (0, 16)	16103.0
  (0, 17)	19852.0
  (0, 18)	0.0552550553383
  (0, 19)	0.105974868936
  (0, 20)	0.775235083632
  (0, 21)	0.0376133810435
  (0, 22)	0.025921611051
  (0, 23)	3.0
  (0, 24)	5.0
  (0, 25)	3411.0
  (0, 26)	2612.0
  (0, 27)	2928.0
  (0, 28)	0.0425095279977
  (0, 29)	0.0454412195837
  (0, 30)	0.807681031955
  (0, 31)	0.0331281149223
  (0, 32)	0.0712401055409
  (0, 33)	4.0
  (0, 34)	3.0


### Input Data is ready for algorithms to process

##### Random Forest with training accuracy

In [144]:
from sklearn.ensemble import RandomForestClassifier

In [145]:
clf = RandomForestClassifier(n_jobs=-1)

In [146]:
%time clf.fit(inputDF,df['Priority'])

CPU times: user 31.1 s, sys: 239 ms, total: 31.3 s
Wall time: 9.49 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [147]:
clf.score(inputDF,df['Priority'])

0.9905588805529798

In [148]:
print 'P1:',clf.score(inputDF[list(df.loc[df.Priority=='P1',:].index)], df['Priority'].loc[df.Priority=='P1'] ),sum(df.Priority=='P1')
print 'P2:',clf.score(inputDF[list(df.loc[df.Priority=='P2',:].index)], df['Priority'].loc[df.Priority=='P2'] ),sum(df.Priority=='P2')
print 'P3:',clf.score(inputDF[list(df.loc[df.Priority=='P3',:].index)], df['Priority'].loc[df.Priority=='P3'] ),sum(df.Priority=='P3')
print 'P4:',clf.score(inputDF[list(df.loc[df.Priority=='P4',:].index)], df['Priority'].loc[df.Priority=='P4'] ),sum(df.Priority=='P4')
print 'P5:',clf.score(inputDF[list(df.loc[df.Priority=='P5',:].index)], df['Priority'].loc[df.Priority=='P5'] ),sum(df.Priority=='P5')

P1: 0.958671301661 2589
P2: 0.953079178886 6138
P3: 0.999550049916 71119
P4: 0.891920251836 1906
P5: 0.882854926299 1289


In [149]:
print 'P1:',clf.score(inputDF[list(df.loc[df.Priority=='P1',:].index)], df['Priority'].loc[df.Priority=='P1'] ),sum(df.Priority=='P1')
print 'P2:',clf.score(inputDF[list(df.loc[df.Priority=='P2',:].index)], df['Priority'].loc[df.Priority=='P2'] ),sum(df.Priority=='P2')
print 'P3:',clf.score(inputDF[list(df.loc[df.Priority=='P3',:].index)], df['Priority'].loc[df.Priority=='P3'] ),sum(df.Priority=='P3')
print 'P4:',clf.score(inputDF[list(df.loc[df.Priority=='P4',:].index)], df['Priority'].loc[df.Priority=='P4'] ),sum(df.Priority=='P4')
print 'P5:',clf.score(inputDF[list(df.loc[df.Priority=='P5',:].index)], df['Priority'].loc[df.Priority=='P5'] ),sum(df.Priority=='P5')

P1: 0.958671301661 2589
P2: 0.953079178886 6138
P3: 0.999550049916 71119
P4: 0.891920251836 1906
P5: 0.882854926299 1289


## Cross validation on the dataset with F1-Score (our evaluation metric)

In [83]:
from sklearn.naive_bayes import MultinomialNB

In [84]:
clf = MultinomialNB()

In [85]:
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [86]:
f1scorer_macro = make_scorer(f1_score, average='macro')
f1scorer_weighted = make_scorer(f1_score, average='weighted')
f1scorer_perclass = make_scorer(f1_score, average=None)

In [87]:
f1scorer_perclass

make_scorer(f1_score, average=None)

In [40]:
%time cvscores_wf1 = cross_val_score(clf, inputDF, df['Priority'], scoring=f1scorer_weighted , cv=10, n_jobs=-1)
print cvscores_wf1
print 'Weighted F1 mean,std:', np.mean(cvscores), np.std(cvscores)

CPU times: user 832 ms, sys: 76 ms, total: 908 ms
Wall time: 2min 4s


In [89]:
import numpy as np

In [90]:
%time cvscores_f1 = cross_val_score(clf, inputDF, df['Priority'], scoring=f1scorer_macro , cv=10, n_jobs=-1)
print cvscores_f1
print 'F1 mean,std:', np.mean(cvscores_f1), np.std(cvscores_f1)

CPU times: user 948 ms, sys: 92 ms, total: 1.04 s
Wall time: 1.61 s
[ 0.19922428  0.2453444   0.29564239  0.292726    0.25145097  0.25054827
  0.25181577  0.24395555  0.22604281  0.21915404]
F1 mean,std: 0.247590448633 0.0283164018714


In [92]:
cvscores_acc = cross_val_score(clf, inputDF, df['Priority'],scoring='accuracy', cv=10, n_jobs=-1)

In [127]:
np.mean(cvscores_acc),np.std(cvscores_acc)

(0.83135657064570512, 0.043975080942427851)

## Measuring Per Class F1 scores as per the paper using KFold RF

In [51]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [52]:
#clf = RandomForestClassifier(n_jobs=-1)
#clf = AdaBoostClassifier()
#clf = GradientBoostingClassifier()
clf = RandomForestClassifier(class_weight="balanced", n_jobs=-1)
kf = KFold(n_splits=10)

In [53]:
cvscores = []
for train,test in kf.split(inputDF):
    train_input = inputDF[train]
    test_input = inputDF[test]
    train_output = df['Priority'][train]
    test_output = df['Priority'][test]
    %time clf.fit(train_input,train_output)
    prediction = clf.predict(test_input)
    scores = f1_score(test_output, prediction, average=None)
    print scores
    cvscores.append(scores)

CPU times: user 24.7 s, sys: 128 ms, total: 24.9 s
Wall time: 5.28 s
[ 0.31662269  0.19651741  0.92780072  0.19827586  0.35359116]
CPU times: user 23.8 s, sys: 40 ms, total: 23.9 s
Wall time: 4.93 s
[ 0.28791774  0.20049505  0.92599266  0.23478261  0.36734694]
CPU times: user 24.2 s, sys: 32 ms, total: 24.2 s
Wall time: 5.06 s
[ 0.30933333  0.21879022  0.93141034  0.31441048  0.36464088]
CPU times: user 24.7 s, sys: 24 ms, total: 24.8 s
Wall time: 5.15 s
[ 0.29878049  0.21957041  0.92621204  0.26356589  0.27380952]
CPU times: user 23.9 s, sys: 32 ms, total: 24 s
Wall time: 5.06 s
[ 0.37533512  0.20595533  0.92771967  0.2519084   0.28205128]
CPU times: user 24.5 s, sys: 4 ms, total: 24.5 s
Wall time: 5.02 s
[ 0.3027027   0.24175824  0.92835264  0.18348624  0.21857923]
CPU times: user 24.1 s, sys: 20 ms, total: 24.1 s
Wall time: 4.92 s
[ 0.31898734  0.20454545  0.9281768   0.25663717  0.27906977]
CPU times: user 24.1 s, sys: 20 ms, total: 24.1 s
Wall time: 4.96 s
[ 0.30594901  0.18457649

In [105]:
clf.feature_importances_[345:357]

array([ 0.02804619,  0.02679268,  0.04025911,  0.02318562,  0.03207399,
        0.02952609,  0.02597535,  0.02539812,  0.03137977,  0.02445142,
        0.02760929,  0.02762818])

In [54]:
np.mean(cvscores)

0.40269769697031571

In [128]:
print '\tP1\t\tP2\t\tP3\t\tP4\tP5'
print '\t',np.mean(cxvscores,axis=0)
cvscores

	P1		P2		P3		P4	P5
	[ 0.25789875  0.15352413  0.92717314  0.19530204  0.26585209]


[array([ 0.302267  ,  0.18350515,  0.92603378,  0.23529412,  0.15525114]),
 array([ 0.26213592,  0.14945055,  0.92646593,  0.17275748,  0.23923445]),
 array([ 0.28078818,  0.16107383,  0.92836827,  0.16153846,  0.27102804]),
 array([ 0.25118483,  0.13333333,  0.92848115,  0.18532819,  0.2815534 ]),
 array([ 0.26633166,  0.14790997,  0.9264115 ,  0.21568627,  0.31884058]),
 array([ 0.22828784,  0.15350389,  0.92756072,  0.22857143,  0.28571429]),
 array([ 0.22429907,  0.16768916,  0.92261082,  0.18181818,  0.27160494]),
 array([ 0.25      ,  0.18441273,  0.92873345,  0.15827338,  0.24210526]),
 array([ 0.26065163,  0.13125   ,  0.92667443,  0.21678322,  0.3286385 ]),
 array([ 0.25304136,  0.12311266,  0.93039138,  0.1969697 ,  0.26455026])]

In [97]:
print '\tP1\t\tP2\t\tP3\t\tP4\tP5'
print '\t',np.mean(cvscores,axis=0)
cvscores

	P1		P2		P3		P4	P5
	[ 0.08830426  0.07866991  0.92049622  0.14514483  0.22485906]


[array([ 0.08044383,  0.04022627,  0.79617796,  0.41274238,  0.45138889]),
 array([ 0.16113744,  0.07654987,  0.84787319,  0.16027875,  0.32773109]),
 array([ 0.12345679,  0.07969349,  0.9126399 ,  0.13333333,  0.30188679]),
 array([ 0.12903226,  0.09279609,  0.94214536,  0.20833333,  0.23809524]),
 array([ 0.11570248,  0.07446809,  0.93965829,  0.1048951 ,  0.35514019]),
 array([ 0.17777778,  0.12815126,  0.93686122,  0.15770609,  0.18918919]),
 array([ 0.032     ,  0.08979592,  0.96259477,  0.09782609,  0.13636364]),
 array([ 0.06349206,  0.06392694,  0.95530839,  0.10084034,  0.16949153]),
 array([ 0.        ,  0.07246377,  0.97441481,  0.03488372,  0.07692308]),
 array([ 0.        ,  0.06862745,  0.93728831,  0.04060914,  0.00238095])]

In [98]:
np.mean(cvscores)

0.29149485632471661

In [96]:
print '\tP1\t\tP2\t\tP3\t\tP4\tP5'
print '\t',np.mean(cvscores,axis=0)
cvscores
#np.mean(cvscores)

	P1		P2		P3		P4	P5
	[ 0.11279759  0.09646663  0.91850115  0.08118626  0.01135761]


[array([ 0.04953999,  0.08259587,  0.79588336,  0.02331002,  0.04402516]),
 array([ 0.22727273,  0.09788769,  0.8428645 ,  0.04580153,  0.02409639]),
 array([ 0.14044944,  0.10748003,  0.90991423,  0.07407407,  0.        ]),
 array([ 0.11034483,  0.12962963,  0.94013304,  0.09195402,  0.        ]),
 array([ 0.10833333,  0.10899873,  0.93713224,  0.11347518,  0.        ]),
 array([ 0.17045455,  0.13226453,  0.93455905,  0.14545455,  0.        ]),
 array([ 0.09230769,  0.12015504,  0.96227744,  0.12765957,  0.04545455]),
 array([ 0.11347518,  0.05504587,  0.95473876,  0.13513514,  0.        ]),
 array([ 0.06451613,  0.07453416,  0.97260002,  0.03529412,  0.        ]),
 array([ 0.05128205,  0.05607477,  0.93490881,  0.01970443,  0.        ])]

### Running algorithms on  Data With Temporal features

In [58]:
import pandas as pd

In [59]:
df = pd.read_csv('../data/data_with_temporal.csv')
df.drop(['Keywords','Summary.1','Status','Resolution','Changed','Opened'],axis=1,inplace=True)
print df.shape
df.dropna(inplace=True)
print df.shape
df.reset_index(inplace=True)
df.isnull().sum()

(103805, 19)
(103802, 19)


index        0
Bug ID       0
Product      0
Component    0
Assignee     0
Summary      0
Priority     0
Severity     0
TMP1         0
TMP2         0
TMP3         0
TMP4         0
TMP5         0
TMP6         0
TMP7         0
TMP8         0
TMP9         0
TMP10        0
TMP11        0
TMP12        0
dtype: int64

In [106]:
df = df.sample(frac=1.0)

In [107]:
df.reset_index(inplace=True)

In [108]:
dummy_df = pd.get_dummies( df[['Product','Component','Assignee','Severity']] )

In [110]:
df.columns[9:]

Index([u'TMP1', u'TMP2', u'TMP3', u'TMP4', u'TMP5', u'TMP6', u'TMP7', u'TMP8',
       u'TMP9', u'TMP10', u'TMP11', u'TMP12'],
      dtype='object')

In [111]:
other_features = pd.concat([dummy_df, df[df.columns[9:]]],axis=1)

In [112]:
other_features.shape

(103802, 357)

In [113]:
other_features.dropna().shape

(103802, 357)

In [114]:
dummy_df.shape

(103802, 345)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/processed/train_processed.csv')

In [10]:
df.loc[0,:]

index                                                0
Bug ID                                           63706
Severity                                        normal
Priority                                            P3
Summary     Invalid dirty indicator after checking out
TMP1                                               823
TMP2                                               614
TMP3                                               712
TMP4                                              2245
TMP5                                              1642
TMP6                                              1955
TMP7                                               121
TMP8                                                78
TMP9                                               100
TMP10                                              242
TMP11                                              172
TMP12                                              203
AUT1                                                 3
AUT2      

In [4]:
df.columns

Index([u'Bug ID', u'Severity', u'Priority', u'Summary', u'TMP1', u'TMP2',
       u'TMP3', u'TMP4', u'TMP5', u'TMP6', u'TMP7', u'TMP8', u'TMP9', u'TMP10',
       u'TMP11', u'TMP12', u'AUT1', u'AUT2', u'AUT3', u'PRO1', u'PRO2',
       u'PRO3', u'PRO4', u'PRO5', u'PRO6', u'PRO7', u'PRO8', u'PRO9', u'PRO10',
       u'PRO11', u'PRO12', u'PRO12.1', u'PRO14', u'PRO15', u'PRO16', u'PRO17',
       u'PRO18', u'PRO19', u'PRO20', u'PRO21', u'PRO22'],
      dtype='object')

In [8]:
df.dropna(inplace=True)
df.isnull().sum()

Bug ID      0
Severity    0
Priority    0
Summary     0
TMP1        0
TMP2        0
TMP3        0
TMP4        0
TMP5        0
TMP6        0
TMP7        0
TMP8        0
TMP9        0
TMP10       0
TMP11       0
TMP12       0
AUT1        0
AUT2        0
AUT3        0
PRO1        0
PRO2        0
PRO3        0
PRO4        0
PRO5        0
PRO6        0
PRO7        0
PRO8        0
PRO9        0
PRO10       0
PRO11       0
PRO12       0
PRO12.1     0
PRO14       0
PRO15       0
PRO16       0
PRO17       0
PRO18       0
PRO19       0
PRO20       0
PRO21       0
PRO22       0
dtype: int64

In [9]:
df.reset_index(inplace=True)

In [13]:
df[['Severity','PRO1','PRO12.1']]

RangeIndex(start=0, stop=83041, step=1)